In [1]:
%pip install pandas numpy scikit-learn xgboost lightgbm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install mysql-connector-python



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Cargamos los datos

In [7]:
import mysql.connector
import pandas as pd

# Crear la conexión a la base de datos
connection = mysql.connector.connect(
    user='unir', 
    password='TFEsoniafranciscomiguel', 
    host='212.227.83.77', 
    database='sakai'
)


query = 'SELECT * FROM sakai_final.modelo_prediccion;'
data = pd.read_sql(query, connection)

C:\Users\PERSONAL\AppData\Local\Temp\ipykernel_6188\3587380882.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, connection)


Verificamos los datos

In [8]:
print(data.head())

                               sakai_id  ind_tiene_contenido  \
0  0e54d523-c9f2-4ac6-b708-795e66354cf0                    1   
1  0ff08a5e-2323-4a6a-ab4d-c89a8325d400                    1   
2  0ff33408-9fc2-4e5d-9662-3f84eb92c021                    1   
3  0c133b9d-e572-4de7-97be-76119b52153f                    1   
4  14304a30-e0db-4d30-ac8a-335c4b2dcf1c                    1   

   ind_entrega_tarea  ind_tiene_notificaciones  ind_notificaciones_mail  \
0                  1                         1                        1   
1                  1                         1                        1   
2                  1                         1                        1   
3                  1                         1                        1   
4                  1                         1                        1   

   ind_tiene_mensajes  ind_lee_mensajes  ind_activ_nuevo_mensaje  \
0                   1                 1                        1   
1                   1       

In [9]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Preprocesamiento

In [10]:
# Definir las columnas que serán nuestras características (features)
features = ['ind_tiene_contenido', 'ind_entrega_tarea', 'ind_tiene_notificaciones',
            'ind_notificaciones_mail', 'ind_tiene_mensajes', 'ind_lee_mensajes',
            'ind_activ_nuevo_mensaje', 'ind_activ_respuesta_mensaje',
            'ind_usa_imagen_oficial', 'ind_mostrar_feed', 'ind_status_online',
            'ind_notificac_examen', 'ind_notificac_tareas', 'ind_conexion', 'num_total_visitas']

# Definir las variables dependientes (targets) para los dos modelos
target_1 = 'ind_al_menos_un_aprobado'
target_2 = 'ind_todo_aprobado'

# Crear los conjuntos de características (X) y etiquetas (y) para los dos targets
X = data[features]
y_1 = data[target_1]
y_2 = data[target_2]

# Dividir en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
from sklearn.model_selection import train_test_split
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y_1, test_size=0.2, random_state=42)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y_2, test_size=0.2, random_state=42)


MODELO 1. RANDOM FOREST

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Entrenar modelo Random Forest para ind_al_menos_un_aprobado
rf_model_1 = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_1.fit(X_train_1, y_train_1)

# Predecir en el conjunto de prueba
y_pred_1 = rf_model_1.predict(X_test_1)

# Evaluación del modelo
print("Accuracy para ind_al_menos_un_aprobado con Random Forest:", accuracy_score(y_test_1, y_pred_1))
print(classification_report(y_test_1, y_pred_1))


Accuracy para ind_al_menos_un_aprobado con Random Forest: 0.9411611559489717
              precision    recall  f1-score   support

           0       0.95      0.07      0.14       243
           1       0.94      1.00      0.97      3598

    accuracy                           0.94      3841
   macro avg       0.94      0.54      0.55      3841
weighted avg       0.94      0.94      0.92      3841



In [16]:
# Entrenar modelo Random Forest para ind_todo_aprobado
rf_model_2 = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_2.fit(X_train_2, y_train_2)

# Predecir en el conjunto de prueba
y_pred_2 = rf_model_2.predict(X_test_2)

# Evaluación del modelo
print("Accuracy para ind_todo_aprobado con Random Forest:", accuracy_score(y_test_2, y_pred_2))
print(classification_report(y_test_2, y_pred_2))

Accuracy para nd_todo_aprobado con Random Forest: 0.576933090341057
              precision    recall  f1-score   support

           0       0.61      0.55      0.58      2013
           1       0.55      0.61      0.58      1828

    accuracy                           0.58      3841
   macro avg       0.58      0.58      0.58      3841
weighted avg       0.58      0.58      0.58      3841



MODELO 2. XGBOOST

In [18]:
import xgboost as xgb

# Entrenar modelo XGBoost para ind_al_menos_un_aprobado
xgb_model_1 = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model_1.fit(X_train_1, y_train_1)

# Predecir en el conjunto de prueba
y_pred_1_xgb = xgb_model_1.predict(X_test_1)

# Evaluación del modelo
print("Accuracy para ind_al_menos_un_aprobado con XGBoost:", accuracy_score(y_test_1, y_pred_1_xgb))
print(classification_report(y_test_1, y_pred_1_xgb))


c:\Users\PERSONAL\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [09:33:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy para ind_al_menos_un_aprobado con XGBoost: 0.9409008070814892
              precision    recall  f1-score   support

           0       0.90      0.07      0.14       243
           1       0.94      1.00      0.97      3598

    accuracy                           0.94      3841
   macro avg       0.92      0.54      0.55      3841
weighted avg       0.94      0.94      0.92      3841



In [19]:
# Entrenar modelo XGBoost para ind_todo_aprobado
xgb_model_2 = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model_2.fit(X_train_2, y_train_2)

# Predecir en el conjunto de prueba
y_pred_2_xgb = xgb_model_2.predict(X_test_2)

# Evaluación del modelo
print("Accuracy para ind_todo_aprobado con XGBoost:", accuracy_score(y_test_2, y_pred_2_xgb))
print(classification_report(y_test_2, y_pred_2_xgb))


c:\Users\PERSONAL\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [09:34:31] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy para ind_todo_aprobado con XGBoost: 0.5771934392085395
              precision    recall  f1-score   support

           0       0.61      0.56      0.58      2013
           1       0.55      0.60      0.58      1828

    accuracy                           0.58      3841
   macro avg       0.58      0.58      0.58      3841
weighted avg       0.58      0.58      0.58      3841



MODELO 3. LIGHTGBM

In [20]:
import lightgbm as lgb

# Entrenar modelo LightGBM para ind_al_menos_un_aprobado
lgb_model_1 = lgb.LGBMClassifier()
lgb_model_1.fit(X_train_1, y_train_1)

# Predecir en el conjunto de prueba
y_pred_1_lgb = lgb_model_1.predict(X_test_1)

# Evaluación del modelo
print("Accuracy para ind_al_menos_un_aprobado con LightGBM:", accuracy_score(y_test_1, y_pred_1_lgb))
print(classification_report(y_test_1, y_pred_1_lgb))

[LightGBM] [Info] Number of positive: 14354, number of negative: 1010
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001992 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 33
[LightGBM] [Info] Number of data points in the train set: 15364, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.934262 -> initscore=2.654078
[LightGBM] [Info] Start training from score 2.654078
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

In [21]:
# Entrenar modelo LightGBM para ind_todo_aprobado
lgb_model_2 = lgb.LGBMClassifier()
lgb_model_2.fit(X_train_2, y_train_2)

# Predecir en el conjunto de prueba
y_pred_2_lgb = lgb_model_2.predict(X_test_2)

# Evaluación del modelo
print("Accuracy para ind_todo_aprobado con LightGBM:", accuracy_score(y_test_2, y_pred_2_lgb))
print(classification_report(y_test_2, y_pred_2_lgb))


[LightGBM] [Info] Number of positive: 7106, number of negative: 8258
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001345 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 33
[LightGBM] [Info] Number of data points in the train set: 15364, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.462510 -> initscore=-0.150243
[LightGBM] [Info] Start training from score -0.150243
Accuracy para ind_todo_aprobado con LightGBM: 0.5782348346784691
              precision    recall  f1-score   support

           0       0.61      0.56      0.58      2013
           1       0.55      0.60      0.58      1828

    accuracy                           0.58      3841
   macro avg       0.58      0.58      0.58      3841
weighted avg       0.58      0.58      0.58      3841



In [23]:
from sklearn.metrics import confusion_matrix

# Matriz de confusión para Random Forest en ind_al_menos_un_aprobado
print("Random Forest en ind_al_menos_un_aprobado:",confusion_matrix(y_test_1, y_pred_1))

# Matriz de confusión para Random Forest en ind_todo_aprobado
print("Random Forest en ind_todo_aprobado:",confusion_matrix(y_test_2, y_pred_2))

# Matriz de confusión para XGBoost en ind_al_menos_un_aprobado
print("XGBoost en ind_al_menos_un_aprobado:",confusion_matrix(y_test_1, y_pred_1_xgb))

# Matriz de confusión para XGBoost en ind_todo_aprobado
print("XGBoost en ind_todo_aprobado:",confusion_matrix(y_test_2, y_pred_2_xgb))

# Matriz de confusión para LightGBM en ind_al_menos_un_aprobado
print("LightGBM en ind_al_menos_un_aprobado:",confusion_matrix(y_test_1, y_pred_1_lgb))

# Matriz de confusión para LightGBM en ind_todo_aprobado
print("LightGBM en ind_todo_aprobado:",confusion_matrix(y_test_1, y_pred_2_lgb))


Random Forest en ind_al_menos_un_aprobado: [[  18  225]
 [   1 3597]]
Random Forest en ind_todo_aprobado: [[1108  905]
 [ 720 1108]]
XGBoost en ind_al_menos_un_aprobado: [[  18  225]
 [   2 3596]]
XGBoost en ind_todo_aprobado: [[1118  895]
 [ 729 1099]]
LightGBM en ind_al_menos_un_aprobado: [[  18  225]
 [   4 3594]]
LightGBM en ind_todo_aprobado: [[ 222   21]
 [1629 1969]]


: 